## Grundlagen Maschineller Lernverfahren | ML_INF19A | 2021
**Datum: 09.11.2021**

#Threshold anpassen

### Daten

In [ ]:
from sklearn.datasets import make_classification

import matplotlib.pyplot as plt
import numpy as np


# Erstelle Datensatz für Klassifikation
# Hinweis: Gewichtung führt dazu, dass der Datensatz ein starkes Ungleichgewicht beinhaltet
# Dokumentation: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html
X, y = make_classification(n_samples=10000, n_features=2, n_redundant=0,
	n_clusters_per_class=1, weights=[0.99, 0.01], flip_y=0, random_state=2021)


In [ ]:
# Zeige Labels
print(y)

In [ ]:
# Wieviele Instanzen welcher Klasse gibt es?
print(np.count_nonzero(y)) # Anzahl Labels mit Klasse 1
print(X.shape[0] - np.count_nonzero(y)) # Anzahl Labels mit Klasse 0

In [ ]:
# Visualisiere
plt.figure()
plt.plot(X[:, 0][y==0], X[:, 1][y==0], "bv")
plt.plot(X[:, 0][y==1], X[:, 1][y==1], "go")
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

# Aufteilen der Datenmenge für Training und Testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2021, stratify=y)

### Modell

In [ ]:
# Bibliothek des Log Reg Modell laden
from sklearn.linear_model import LogisticRegression

# Log Reg Modell erstellen und trainieren
model = LogisticRegression()
model.fit(X_train, y_train)

In [ ]:
# Vorhersage / Vorschläge des Modells errechnen für die Testdaten
y_test_pred = model.predict(X_test) # Standard Threshold -> z >= 0.5 (vgl. Vorlesung)

In [ ]:
# Ermittle Performancewerte
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

print("Accuracy:\t", accuracy_score(y_test, y_test_pred))
print("Precision:\t", precision_score(y_test, y_test_pred))
print("Recall:\t\t", recall_score(y_test, y_test_pred))
print("F1 Score:\t", f1_score(y_test, y_test_pred))

In [ ]:
# Welche Wahrscheinlichkeiten für welche Klasse hat das Modell bestimmt?
y_test_pred_proba = model.predict_proba(X_test)

# Dimension: (Instanzen, #Klassen)
y_test_pred_proba.shape

In [ ]:
# Zeige Klassenwahrscheinlichkeiten für jede Instanz an
print(y_test_pred_proba) # (Klasse 0 (neg), Klasse 1 (pos))

In [ ]:
# Zeige, dass Default Threshold >= 0.5 ist
y_test_pred_threshold = (model.predict_proba(X_test)[:,1] >= 0.5).astype(int)

In [ ]:
# Prüfen ob Ergebnisse identisch sind
assert np.all(y_test_pred == y_test_pred_threshold)

### Performance

In [ ]:
from sklearn.metrics import confusion_matrix

# Confusion Matrix berechnen
conf_matrix = confusion_matrix(y_test, y_test_pred)
print(conf_matrix)

In [ ]:
# Extrahiere nur die Wahrscheinlichkeiten für die POSITIVE Klasse (hier also Label = 1)
y_test_pred_proba_class1 = y_test_pred_proba[:, 1]

In [ ]:
from sklearn.metrics import roc_curve

# Berechne die ROC Kurve (Grafische Darstellung der PErformance eines binären Klassifikators)
# https://scikit-learn.org/stable/modules/generated/sklearn.metrics.roc_curve.html
fpr, tpr, thresholds = roc_curve(y_test, y_test_pred_proba_class1)

In [ ]:
# Zeichne die ROC Kurve
plt.figure()
plt.plot([0,1], [0,1], linestyle='--', label='Zufall')
plt.plot(fpr, tpr, marker='.', label='Log. Reg.')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()

### Geometrisches Mittel

In [ ]:
# Berechne geometrisches Mittel für alle Thresholds der ROC Kurve
gmeans = np.sqrt(tpr * (1-fpr)) # https://de.wikipedia.org/wiki/Geometrisches_Mittel

# locate the index of the largest g-mean
# Identifiziere den größten Wert des geom. Mittels
idx = np.argmax(gmeans)
print('Bester Threshold=%.5f, Geometrisches Mittel=%.5f' % (thresholds[idx], gmeans[idx]))

In [ ]:
# Zeichne besten Threshold ein (über Werte von FPR und TPR bei diesem Wert)
plt.figure()
plt.plot([0,1], [0,1], linestyle='--', label='Zufall')
plt.plot(fpr, tpr, marker='.', label='Log. Reg.')
plt.scatter(fpr[idx], tpr[idx], marker='o', color='black', label='Best')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()

## Modellverhalten bei angepassten  Thresholds

In [ ]:
# Aufbau der Confusion Matrix
# TN  FP
# FN  TP

# Erinnerung:
#############
# Precision = TP / (TP + FP) --> Wieviele Ergebnisse sind "sinnvoll""? (d.h. der Anteil der TP zu den als POSITIVE ermittelten)
# Recall = TP / (TP + FN)    --> Wieviele "sinnvolle" Ergebnisse wurden ermittelt? (d.h. der Anteil der TP zu den wirklich POSITIVEN)

### Threshold = 0.5 (Standard)

In [ ]:
# ERINNERUNG: So würde das Stadardmodell vorhersagen
####################################################

# Standard Threshold = 0.5 (Referenz)
# Confusion Matrix berechnen
conf_matrix = confusion_matrix(y_test, y_test_pred)
print(conf_matrix)

print("Accuracy:\t", accuracy_score(y_test, y_test_pred))
print("Precision:\t", precision_score(y_test, y_test_pred))
print("Recall:\t\t", recall_score(y_test, y_test_pred))
print("F1 Score:\t", f1_score(y_test, y_test_pred))

In [ ]:
# Erinnerung: Datensatz beinhaltet sehr wenige "POSITIVE" Klassen (-> 100), aber sehr viele "NEGATIVE" (->9900)
# Ziel ist es auch möglichst viele "POSITIVE" vorzuschlagen

### Threshold = 0.015

In [ ]:
# Setze Threshold auf 0.015 (~ bester berechneter Wert laut ROC Kurve, vgl. oben)
y_test_pred_threshold = (model.predict_proba(X_test)[:,1] >= 0.015).astype(int)
conf_matrix_threshold = confusion_matrix(y_test, y_test_pred_threshold)
print(conf_matrix_threshold)

# Ergebnis: 
#############
# - weniger TN
# - mehr TP
# - weniger FN (unterbliebender Alarm)
# - mehr FP (Fehlalarme)

# -> Tendenz geht dahin, mehr Instanzen der "POSITIVEN" Klasse zuzuordnen (was auch gewünscht ist)

In [ ]:
print("Accuracy:\t", accuracy_score(y_test, y_test_pred_threshold))
print("Precision:\t", precision_score(y_test, y_test_pred_threshold))
print("Recall:\t\t", recall_score(y_test, y_test_pred_threshold))
print("F1 Score:\t", f1_score(y_test, y_test_pred_threshold))

### Threshold = 0.99

In [ ]:
# Setze Threshold auf 0.99
y_test_pred_threshold = (model.predict_proba(X_test)[:,1] >= 0.99).astype(int)
conf_matrix_threshold = confusion_matrix(y_test, y_test_pred_threshold)
print(conf_matrix_threshold)

# Ergebnis: 
#############
# - weniger TN
# - weniger TP
# - mehr FN (unterbliebender Alarm)
# - gleich viele FP (Fehlalarme)

# -> Tendenz geht dahin, mehr Instanzen der "NEGATIVEN" Klasse zuzuordnen

In [ ]:
print("Accuracy:\t", accuracy_score(y_test, y_test_pred_threshold))
print("Precision:\t", precision_score(y_test, y_test_pred_threshold))
print("Recall:\t\t", recall_score(y_test, y_test_pred_threshold))
print("F1 Score:\t", f1_score(y_test, y_test_pred_threshold))

### Threshold = 0.0005

In [ ]:
# Setze Threshold = 0.0005
y_test_pred_threshold = (model.predict_proba(X_test)[:,1] >= 0.0005).astype(int)
conf_matrix_threshold = confusion_matrix(y_test, y_test_pred_threshold)
print(conf_matrix_threshold)

# Ergebnis: 
#############
# - viel weniger TN
# - mehr TP
# - weniger FN (unterbliebender Alarm)
# - viel mehr FP (Fehlalarme)


# -> Tendenz geht dahin, sehr viel mehr Instanzen der "POSITIVEN" Klasse zuzuordnen

In [ ]:
print("Accuracy:\t", accuracy_score(y_test, y_test_pred_threshold))
print("Precision:\t", precision_score(y_test, y_test_pred_threshold))
print("Recall:\t\t", recall_score(y_test, y_test_pred_threshold))
print("F1 Score:\t", f1_score(y_test, y_test_pred_threshold))

In [ ]:
# HINWEIS: Nun können auch analog Accuracy, Precision, Recall und F1 Performancewerte berechnen werden.
# Dies wird als "Hausaufgabe" empfohlen.